In [1]:
import numpy as np
import sys
import nest_asyncio
import math 
from geopy import distance
from math import cos
from haversine import Unit
from pyproj import Proj
import pyproj
from math import cos, radians

#import rasterio
import random
from natsort import natsorted,realsorted, ns
from tqdm.notebook import trange, tqdm
from time import sleep
import osmnx as ox
import haversine as hs
import shutil
from numpy import asarray
from IPython import display
import os
from os import listdir
import utm
import geopandas
from pathlib import Path
import glob
import concurrent.futures
import geotiler

import matplotlib.pyplot as plt
import h5py
import imutils
import mplleaflet
from PIL import Image, ImageDraw
print(mplleaflet.__file__)

c:\users\bjqb7h\miniconda3\envs\perl\lib\site-packages\mplleaflet\__init__.py


In [2]:
Logname = 'AtCityBMW_Applanix-20220228T141708Z393'
path = 'C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/GPS DATA/'
path_position = 'sensors/ApplanixDGPS'
path_orientation = 'sensors/ApplanixDGPS/orientation'
path_timestamps = 'sensors/ApplanixDGPS/timestamps'
path_meta = 'C:/Users/bjqb7h/Downloads/Thesis2022/Meta/New Meta/'
path_save ='C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_1/Dataset'
Rdirectory =r'C:\Users\bjqb7h\Downloads\Thesis2022\HD Maps_Rotate'
path_rotate = 'C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_1/Dataset/'
path_crop ='C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_Rotate'
path_cropped ='C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_Cropped'
Sep_path_save ='C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_1/Dataset/RadarGPS/'
Sep_Rdirectory = r'C:\Users\bjqb7h\Downloads\Thesis2022\HD Maps_Rotate\GPSRADAR\RotatedMAPS'
Sep_path_Rotate = r'C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_1/Dataset/RadarGPS'
Crop_directory= r'C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_Rotate/GPSRADAR/RotatedMAPS'
Crop_save_path =r'C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_Cropped/GPSRADAR'
hDF5_PATH = 'C:/Users/bjqb7h/Downloads/Thesis2022/NUMPY/New'
dir0 = r'C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_1/Dataset/RadarGPS'
dir1 = 'C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_Rotate/GPSRADAR/RotatedMAPS'
dir2 = 'C:/Users/bjqb7h/Downloads/Thesis2022/HD Maps_Cropped/GPSRADAR'



In [33]:


#Read Hdf5 filel for the Radar data and GPS data.
hf1 = h5py.File(path+Logname+'.h5','r')
dset = hf1[path_position]['position']
dset1= hf1[path_orientation]
dsetA = hf1.get(path_timestamps)
GPSTimestamp=np.array(dsetA)



hf2 = h5py.File(path_meta+Logname+'.h5','r')
dsetB= hf2.get('radar_timestamps')
dsetB=np.array(dsetB)
Radar1Timestamp = dsetB[:,0]
Radar1Timestamp = np.round(Radar1Timestamp,2)#RadarTime stamps 
GPSTimestamp= np.round(GPSTimestamp,2)#GPS

Noise_added= 0.0000150 #Range -0.0000150<Noise<0.0000150



print(GPSTimestamp.shape,Radar1Timestamp.shape)

(979,) (204,)


In [30]:



#Extract all the Latitudes,Longitude and Orientation(Quaternions) 
Long=dset[:,0] #Longitude
Lat=dset[:,1]  #Latitude
q0=dset1[:,0]  #Quaternions 1
q1=dset1[:,1]  #Quaternions 2
q2=dset1[:,2]  ##Quaternions 3
q3=dset1[:,3] ##Quaternions 4

size= len(Long) #Size of the dataset ususally len(long)

nest_asyncio.apply()
plt.plot(Long,Lat,'-g',linewidth=10)
mplleaflet.display()

In [9]:
def plot_gps_data(logpath, logname, output_dir):
    # Read H5 file
    with h5py.File(os.path.join(logpath, logname), 'r') as hf1:
        dset = hf1[path_position]['position']
        Long = dset[:, 0]  # Longitude
        Lat  = dset[:, 1]  # Latitude

    # Plot the data
    plt.plot(Long, Lat, '-g', linewidth=1)
    
    # Save the map as an HTML file with the log name in the specified output directory
    figure_filename = os.path.join(output_dir, os.path.splitext(logname)[0] + '.html')
    mplleaflet.show(path=figure_filename)

    print('Map saved:', figure_filename)

# Path to the folder containing the logs
logpath = 'C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/GPS DATA/'

# Specify the output directory for the maps
output_dir = 'C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/'

# Iterate over files in the folder
for filename in os.listdir(logpath):
    if filename.endswith('.h5'):
        plot_gps_data(logpath, filename, output_dir)

Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220221T160319Z085.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220221T160419Z905.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220221T160443Z413.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220221T160603Z161.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220221T160706Z153.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220221T160814Z341.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220221T161036Z537.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220221T161110Z753.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW

Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220228T153921Z061.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220228T154110Z730.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220228T154254Z933.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220228T154344Z309.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220228T154511Z217.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220228T154619Z217.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220228T154717Z377.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220228T154849Z253.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW

Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220318T095113Z773.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220318T095301Z181.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220318T095448Z233.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220318T095644Z089.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220318T095843Z354.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220318T100037Z829.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220318T100225Z090.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220318T100433Z841.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW

Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220323T100030Z743.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220323T100220Z293.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220323T100409Z501.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220323T100600Z717.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220323T100749Z653.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220323T100946Z153.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220323T101138Z893.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW_Applanix-20220323T101332Z493.html
Map saved: C:/Users/bjqb7h/Downloads/Thesis2022/GPS DATA/colored plots/AtCityBMW

In [ ]:
np.random.seed(137)
percentage = 0.001
Longn = np.random.normal(0.03, Long.std(), Long.size) * percentage
Latn = np.random.normal(0.03, Lat.std(), Lat.size) * percentage
Long_n = Long + Longn
Lat_n = Lat + Latn

fig = plt.figure()

ax1 = fig.add_subplot(211)
ax1.set_title('Noise added to Longitude signal')
ax1.plot(Long,'-', linewidth=1,label='pure signal')
ax1.plot(Long_n,'-g',linewidth=1, label='signal+noise')
plt.legend()


ax2 = fig.add_subplot(212)
ax2.plot(Long_n - Long, label='added noise', c='r')
plt.legend()
fig = plt.figure()

ax1 = fig.add_subplot(211)
ax1.set_title('Noise added to Latitude part of the signal')
ax1.plot( Lat, label='pure signal')
random_indices = np.random.randint(0, Long.size, int(Long.size*percentage))

pr = Long.copy()
pr[random_indices] += Lat_n[random_indices]
ax1.plot(Lat_n, label='signal+noise')

ax2 = fig.add_subplot(212)
ax2.plot( Lat_n - Lat, label='added noise', c='r')
plt.legend()
plt.show()

In [ ]:
plt.plot(Long_n,Lat_n)

In [ ]:
#plt.plot(Long_n, Lat_n, '-r',linewidth=1)
plt.plot(Long, Lat, '-g',linewidth=1)
mplleaflet.display()

In [ ]:
#checking if the distance added is the correct in Lat and long

# from math import radians, cos, sin, asin, sqrt
# def haversine(lon1, lat1, lon2, lat2):
#     """
#     Calculate the great circle distance in kilometers between two points 
#     on the earth (specified in decimal degrees)
#     """
#     # convert decimal degrees to radians 
#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

#     # haversine formula 
#     dlon = lon2 - lon1 
#     dlat = lat2 - lat1 
#     a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
#     c = 2 * asin(sqrt(a)) 
#     r = 6378137 #6371 for Km # Radius of earth in meters. Use 3956 for miles. Determines return value units.
#     return c * r



#The factor between the above generated New Lat and New Long and the Haversine distance is 1.0823265
#eg if above we specify the distance as 10 meters the Haversine distance would be 10*1.0823265 = 10.823265 meters

In [ ]:
TestLat= Lat+np.random.normal(0,Lat.std()/100,Lat.size)
TestLong = Long+np.random.normal(0,Long.std()/100,Long.size)
plt.plot(TestLong,TestLat,'-b',linewidth=1)
mplleaflet.display()
#plt.plot(Long, Lat, '-r',linewidth=1)


In [ ]:
utm_proj = Proj(proj='utm', zone=32, ellps='WGS84')
x, y = utm_proj(Long, Lat)
#print(x.size,y.size)

#To add noise of up to 5 meters to the longitude and latitude dimensions,
#we can set the variance (sigma^2) for each dimension to 25, 
#since the variance is equal to the square of the standard deviation, and the standard deviation is equal to 5 meters. 
#The standard deviation is a measure of the spread of the data around the mean, and in this case,
#we want the noise to be within a range of +/- 5 meters around the original GPS coordinates.
def add_gaussian_noise(x,y):
    noise = np.random.multivariate_normal(mean=[0, 0], cov=[[6, 0], [0, 6]], size=x.shape)
    x_noisy = x + noise[:, 0]
    y_noisy = y + noise[:, 1]
    return x_noisy, y_noisy

NLong,NLat = add_gaussian_noise(x,y)
longitude, latitude = utm_proj(NLong, NLat, inverse=True)
plt.plot(longitude, latitude,'-b')
plt.plot(Long,Lat,'-r')
mplleaflet.display()



In [ ]:
# Calculate the noise in the longitude and latitude dimensions
noise_longitude = longitude - Long
noise_latitude = latitude - Lat
plt.hist(noise_longitude, bins=40, density=False)
plt.hist(noise_latitude, bins=40, density=False)
plt.xlabel("Noise range")
plt.ylabel("frequency of the noise ")
plt.title("Noise representation for latitude and longitude")
plt.show()
#the blue curve shows the distribution of the noise in the longitude dimension 
#and the orange curve shows the distribution of the noise in the latitude dimension.

In [ ]:

fig, ax = plt.subplots(1, 2)
noise = np.random.multivariate_normal(mean=[0, 0], cov=[[25, 0], [0, 25]], size=x.shape)
# Plot a histogram of the noise in the longitude dimension
ax[0].hist2d(noise[:,0], noise[:, 1], bins=(100,100), density=True)


# # Update the x-axis label to reflect the unit (degrees)
# ax[0].set_xlabel('Noise (degrees)')
# ax[0].set_title('Noise in Longitude Dimension')

# # Plot a histogram of the noise in the latitude dimension
# ax[1].hist(noise_latitude, bins=50, density=False)

# # Update the x-axis label to reflect the unit (degrees)
# ax[1].set_xlabel('Noise (degrees)')
# ax[1].set_title('Noise in Latitude Dimension')

# Show the plots
plt.show()


In [ ]:
fig = plt.figure()
ax = fig.subplots(1, 2)

# Plot a histogram of the noise in the longitude dimension
ax[0].hist(noise_longitude, bins=50, density=False)

# Get the current x-axis values (in degrees)
x_values = ax[0].get_xlim()

# Convert x-axis values to meters
x_values_meters = [g * 111319 for g in x_values]

# Set the x-axis limits to the converted values
ax[0].set_xlim(x_values_meters)
print("The values for Longitude in meters are ",x_values_meters)


# Update the x-axis label to reflect the new unit
ax[0].set_xlabel('Noise (meters)')

# Plot a histogram of the noise in the latitude dimension
ax[1].hist(noise_latitude, bins=50, density=False)

# Get the current x-axis values (in degrees)
y_values = ax[1].get_xlim()

# Convert x-axis values to meters
y_values_meters = [y * 111319 for y in y_values]

# Set the x-axis limits to the converted values
ax[1].set_xlim(y_values_meters)
print("The values for Latitude in meters are ",y_values_meters)

# Update the x-axis label to reflect the new unit
ax[1].set_xlabel('Noise (meters)')

# Show the plots
plt.show()

In [ ]:
dx = np.load(r'C:\Users\bjqb7h\Downloads\Thesis2022\dxdy\dx.npy')
dy= np.load(r'C:\Users\bjqb7h\Downloads\Thesis2022\dxdy\dy.npy')
print(dx.shape,dy.shape)
plt.plot(dx,dy)

In [ ]:
org_lat =np.zeros(2093,dtype =np.float64)
org_long =np.zeros(2093,dtype =np.float64)
gps_lat =np.zeros(2093,dtype =np.float64)
gps_long =np.zeros(2093,dtype =np.float64)

gps_timestamps = GPSTimestamp
radar_timestamps = Radar1Timestamp

# use numpy's searchsorted function to find the closest radar timestamps to the GPS timestamps
closest_radar_indices = np.searchsorted(radar_timestamps, gps_timestamps, side='left')

# create an empty list to store the synchronized gps timestamps
gps_sync_index = []

# iterate through the closest radar indices
for i, closest_index in enumerate(closest_radar_indices):
    # check if the closest index is outside of the radar timestamp array bounds
    if closest_index == 0:
        closest_radar_ts = radar_timestamps[0]
    elif closest_index == len(radar_timestamps):
        closest_radar_ts = radar_timestamps[-1]
    else:
        # if the current GPS timestamp is greater than the radar timestamp, take the radar_timestamp before
        if abs(radar_timestamps[closest_index] - gps_timestamps[i]) > abs(radar_timestamps[closest_index - 1] - gps_timestamps[i]):
            closest_radar_ts = radar_timestamps[closest_index - 1]
        else:
            closest_radar_ts = radar_timestamps[closest_index]
            
    if closest_radar_ts == gps_timestamps[i]:
        gps_sync_index.append(i)
#print(len(gps_sync_index))
org_lat = Lat[gps_sync_index]
org_long = Long[gps_sync_index]
gps_lat =latitude[gps_sync_index]
gps_long =longitude[gps_sync_index]
plt.plot(gps_long,gps_lat,'-b')
plt.plot(org_long,org_lat,'-r')
mplleaflet.display()


In [ ]:
#We calculate the number of meters per pixel by dividing the size of the area represented by the image (in meters) by the size of the image (in pixels).
#Then, we use the Haversine formula to calculate the number of degrees per pixel, based on the latitude of the image center.
img_size = (160, 160)
img_area_size = 160
# calculate the number of meters per pixel
# assuming that the image is square
meters_per_pixel = img_area_size / img_size[0]

degrees_per_pixel= np.zeros(2093,dtype =np.float64)
newLat1 =np.zeros(2093,dtype =np.float64)
newLong1 =np.zeros(2093,dtype =np.float64)

for i in range(2093):
    # calculate the number of degrees per pixel
    # using the haversine formula
    degrees_per_pixel[i] = (2 *meters_per_pixel) / (6371 * 1000 * cos(radians(gps_lat[i])))
    dLongitude = (dx[i] * degrees_per_pixel[i]) 
    dLatitude = (dy[i] * degrees_per_pixel[i])
    newLat1[i] = gps_lat[i]+dLatitude
    newLong1[i] = gps_long[i]+dLongitude


In [ ]:
plt.plot(gps_long,gps_lat,'-b')
plt.plot(newLong1,newLat1,'-r')
mse_meters = calculate_mse_meters(org_lat,org_long,newLat1,newLong1)
mse_meters1 = calculate_mse_meters(org_lat,org_long,gps_lat,gps_long)
print(mse_meters)
print(mse_meters1)

In [ ]:
def convert_pixels_to_gps(longitude, latitude, dx, dy, image_width, image_height):
    Earth_radius = 6371000 # meters
    dLongitude = (dx / image_width) * (360 / (2 * math.pi * Earth_radius * math.cos(math.radians(latitude))))
    dLatitude = (dy / image_height) * (360 / (2 * math.pi * Earth_radius))
    new_longitude = longitude + dLongitude
    new_latitude = latitude + dLatitude
    return new_longitude, new_latitude

def calculate_mse(original_latitude, original_longitude, new_latitude, new_longitude):
    mse = 0
    for x1, y1, x2, y2 in zip(original_latitude, original_longitude, new_latitude, new_longitude):
        mse += (x1 - x2) ** 2 + (y1 - y2) ** 2
    mse = mse / len(original_latitude)
    return mse

def calculate_mse_meters(original_latitude, original_longitude, new_latitude, new_longitude):
    mse = 0
    for x1, y1, x2, y2 in zip(original_latitude, original_longitude, new_latitude, new_longitude):
        lat1, lon1, lat2, lon2 = map(math.radians, [x1, y1, x2, y2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
        c = 2 * math.asin(math.sqrt(a))
        m = 6371000 * c
        mse += m ** 2
    mse = mse / len(original_latitude)
    return mse

def calculate_sse_meters(original_latitude, original_longitude, new_latitude, new_longitude):
    sse = 0
    for x1, y1, x2, y2 in zip(original_latitude, original_longitude, new_latitude, new_longitude):
        lat1, lon1, lat2, lon2 = map(math.radians, [x1, y1, x2, y2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
        c = 2 * math.asin(math.sqrt(a))
        m = 6371000 * c
        sse += m ** 2
    return sse



In [ ]:
newLong = np.zeros(2093,dtype =np.float64)
newLat = np.zeros(2093,dtype =np.float64)
for i in range(len(newLong)):
    newLong[i],newLat[i]= convert_pixels_to_gps(gps_long[i],gps_lat[i],dx[i],dy[i],image_width=160,image_height=160)
  

In [ ]:
mse = calculate_mse(org_lat,org_long,newLat,newLong)
mse1 = calculate_mse(org_lat,org_long,gps_lat,gps_long)
mse_meters = calculate_mse_meters(org_lat,org_long,newLat,newLong)
mse_meters1 = calculate_mse_meters(org_lat,org_long,gps_lat,gps_long)
sse_meters = calculate_sse_meters(org_lat,org_long,newLat,newLong)
sse_meters1 = calculate_sse_meters(org_lat,org_long,gps_lat,gps_long)


print("Mean Squared Error b/w DGPS position and GPS position translated by MI :", mse)
print("Mean Squared Error b/w DGPS position and GPS position  :", mse1)
print("Mean Squared Error in meters  :", mse_meters)
print("Mean Squared Error in meters  :", mse_meters1)
print("Sum Squared Error in meters DGPS vs MI :", sse_meters)
print("Sum Squared Error in meters DGPS vs GPS :", sse_meters1)

In [ ]:
plt.plot(gps_long,gps_lat,'-b')
plt.plot(newLong,newLat,'-r')
mplleaflet.display()


In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371e3 # Earth's radius
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2) * math.sin(delta_phi / 2) + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) * math.sin(delta_lambda / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [ ]:
image_width = 160
latitude_range =np.zeros((len(dx_degrees)),dtype =np.float64)
longitude_range =np.zeros((len(dx_degrees)),dtype =np.float64)
new_lat =np.zeros((len(dx_degrees)),dtype =np.float64)
new_long =np.zeros((len(dx_degrees)),dtype =np.float64)
new_lat1 =np.zeros((len(dx_degrees)),dtype =np.float64)
new_long1 =np.zeros((len(dx_degrees)),dtype =np.float64)
latitude_shift = np.zeros((len(dx_degrees)),dtype =np.float64)
longitude_shift= np.zeros((len(dx_degrees)),dtype =np.float64)
for i in range(len(gps_lat)-1):
    latitude_range[i] = gps_lat[i+1]-gps_lat[i]
    longitude_range[i] = gps_long[i+1]-gps_long[i]
    
for i in range(len(gps_lat)):
    pixel_per_degree_latitude[i] = image_width*latitude_range[i]
    pixel_per_degree_longitude[i] = image_width*longitude_range[i]
    latitude_shift[i] = dy[i]/pixel_per_degree_latitude[i]
    longitude_shift[i] = dx[i]/pixel_per_degree_longitude[i]

for i in range(len(gps_lat)):
    #new_lat[i] = org_lat[i]-latitude_shift[i]
    #new_long[i] = org_long[i]-longitude_shift[i]
    new_lat1[i] = gps_lat[i]+latitude_shift[i]*0.5
    new_long1[i] = gps_long[i]+longitude_shift[i]*0.5
    
    


plt.plot(gps_long,gps_lat,'g')
#plt.plot(new_long1,new_lat1,'r')

#plt.plot(org_long,org_lat,'b')
#plt.plot(new_lat[0:100],'r')
#plt.plot(gps_lat[0:100],'b')
#plt.plot(org_lat[0:100],'g')
#plt.plot(longitude_shift,latitude_shift)
print(longitude_shift[0:5],gps_lat[0:5])


MSEERROR= ((org_long - gps_long)**2 + (org_lat - gps_lat)**2).mean()
TERRORPLUS = ((org_long - new_long1)**2 + (org_lat - new_lat1)**2).mean()
#TERRORMINUS = ((org_long - new_long)**2 + (org_lat - new_lat)**2).mean()
#plt.bar(['GPS','Translated GPS'], [MSEERROR*111319*10000, TERRORPLUS*111319*10000])

In [ ]:
#plt.plot(org_long,org_lat,'r')
plt.plot(Long,Lat,'b')
mplleaflet.display()

In [ ]:
#Calculate SSE

def sse(longitude1, latitude1, longitude2, latitude2):
    n = len(longitude1)
    error = 0
    for i in range(n):
        error += (longitude1[i] - longitude2[i])**2 + (latitude1[i] - latitude2[i])**2
    return error

GPSERROR = sse(org_long,org_lat,gps_long,gps_lat)
TRANSERROR = sse(org_long,org_lat,new_long,new_lat)

plt.bar(['GPS','Translated GPS'], [GPSERROR, TRANSERROR])
plt.ylabel('SSE')
plt.show()

In [ ]:
#Alternate

# wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')
# gps_points_meters = pyproj.transform(wgs84, wgs84, gps_long, gps_lat)
# dgps_points_meters = pyproj.transform(wgs84, wgs84, org_long,org_lat)
# # translated_points = pyproj.transform(wgs84, wgs84, new_long, new_lat)



# # calculate the MSE for GPS and DGPS
# squared_errors = [(gps_points_meters[i][0] - dgps_points_meters[i][0]) ** 2 +
#                   (gps_points_meters[i][1] - dgps_points_meters[i][1]) ** 2
#                   for i in range(len(gps_points_meters))]

# # calculate the MSE for TGPS and DGPS
# squared_errors_T = [(translated_points[i][0] - dgps_points_meters[i][0]) ** 2 +
#                   (translated_points[i][1] - dgps_points_meters[i][1]) ** 2
#                   for i in range(len(translated_points))]
squared_errors = (org_long - gps_long) ** 2 +(org_lat - gps_lat) ** 2
squared_errors_T = (org_long - new_long) ** 2 +(org_lat - new_lat) ** 2
                  


mse = sum(squared_errors) / len(gps_long)
mseT = sum(squared_errors_T) / len(org_long)
print(mse)
print(mseT)

In [ ]:
error = [(gps_points_meters[i][0]-dgps_points_meters[i][0],gps_points_meters[i][1]-dgps_points_meters[i][1]) for i in range(len(gps_points_meters))]

x_err = [err[0] for err in error]
y_err = [err[1] for err in error]

plt.hist(x_err, bins=50, color='b', alpha=0.5, label='X error')
plt.hist(y_err, bins=50, color='r', alpha=0.5, label='Y error')
plt.legend()
plt.xlabel('Error (m)')
plt.ylabel('Frequency')
plt.title('Error Distribution')
plt.show()

In [ ]:
#plt.plot(new_long,new_lat,'b-')

plt.plot(longitude,latitude,'r-')
plt.plot(Long,Lat,'b-')
mplleaflet.display()


In [ ]:
n = len(gps_lat)
error_sum = 0
error_tran =0 
utm_proj = Proj(proj='utm', zone=32, ellps='WGS84')
x, y = utm_proj(org_long, org_lat) #DGPS 
x1,y1 = utm_proj(gps_long, gps_lat)
x2,y2 = utm_proj(new_long, new_lat)

for i in range(n):
    error_sum += abs(x[i] - x1[i]) + abs(y[i] - y1[i])
    error_tran += abs(x[i] - x2[i]) + abs(y[i] - y2[i])
    
MAE = error_sum / n
MAET = error_tran / n 
print(MAE)
print(MAET)
plt.bar(['GPS','Translated GPS'], [MAE, MAET])
plt.ylabel('MAE')
plt.show()

In [ ]:
#Add constant noise to the GPS data
#np.random.seed(1)
#np.random.normal The probability density function of the normal distribution
#percentage = 0.02
#Noise Lat and long
def Add_Noise(Long,Lat,Noise=0.0000150):
    #Noise_add=-0.0000150#  Max value {0.00002>0>-0.00002}
    #n_Long = np.arange(Long[0], Long.size, 0.0000001)
    #n_Lat =  np.arange(0, 0.0000001, Lat.size)
    n_Long = np.full(Long.size,Noise)
    n_Lat =  np.full(Lat.size,Noise)
    #New Values of Longitude and Latitude after adding noise 
    Long_n = Long + n_Long
    Lat_n = Lat + n_Lat
    return Long_n,Lat_n


def quat2eulers(q0:float, q1:float, q2:float, q3:float):
    """
    Compute yaw-pitch-roll Euler angles from a quaternion.
    
    Args
    ----
        q0: Scalar component of quaternion.
        q1, q2, q3: Vector components of quaternion.
    
    Returns
    -------
        (roll, pitch, yaw) (tuple): 321 Euler angles in radians
        
    """
    #roll = math.atan2(2 * ((q2 * q3) + (q0 * q1)),q0**2 - q1**2 - q2**2 + q3**2)  # radians
    #pitch = math.asin(2 * ((q1 * q3) - (q0 * q2)))
    yaw = math.atan2(2 * ((q1 * q2) + (q0 * q3)),q0**2 + q1**2 - q2**2 - q3**2)
    degree = 180/math.pi
    yaw = yaw *degree
    return yaw

def CreateAngle(q0,q1,q2,q3,size=size):
    Angle = np.zeros(size,dtype=float)
    for i in range(size):
        Angle[i]=quat2eulers(q0[i],q1[i],q2[i],q3[i])
    return Angle

def BBOX_Generation(Lat,Long):
    padding_Lat =0.001750
    Padding_long=0.001750
    bbox = (min(Lat)-padding_Lat,max(Lat)+padding_Lat,min(Long)-Padding_long, max(Long)+Padding_long)
    gr = ox.graph_from_bbox(*bbox, network_type='all')
    geo = ox.geometries_from_bbox(*bbox,tags={'building':True})
    return gr,geo,bbox

def Convert_To_M(Lat,Long):
    X,Y,O,P=utm.from_latlon(Lat, Long)
    X1=X+80#80 meter to match the semseg grid
    Y1=Y+80
    NewLat,NewLong=utm.to_latlon(X1, Y1,O,P)
    diff_Lat = NewLat-Lat
    diff_Long = NewLong-Long
    return diff_Long,diff_Lat

def get_fig(bbox=None, size_inches=(15,15)):
    fig, ax = plt.subplots(frameon="False")
    fig.set_size_inches(*size_inches)
    ax.set_facecolor("green")
    plt.close(fig)
    ox.plot.plot_figure_ground(GRoad, node_size=0, network_type='drive_service',ax=ax, edge_color="black", bgcolor='blue', bbox=bbox,default_width=18)
    #ox.folium.plot_graph_folium(gr,popup_attribute=None, tiles='cartodbpositron', zoom=1, fit_bounds=True)
    ox.plot.plot_footprints(GBuilding, ax=ax, bgcolor='green', bbox=bbox)
    return fig



def Save_Maps(size,Lat,Long,DIFF_LAT,DIFF_LONG):
    for i in trange(size,desc='Creating images'): #metres to latitude
        North = Lat[i]-DIFF_LAT                           #0.00121
        South = Lat[i]+DIFF_LAT                             #0.0008
        East  = Long[i]-DIFF_LONG                            #0.00121
        West  = Long[i]+DIFF_LONG                             #0.0017
        fig=get_fig(bbox=(North,South,East,West),size_inches=(6,6))
        fig.savefig(path_save +'/SaveMAP%d.png'%i, bbox_inches='tight', pad_inches = 0)
        print("Successfully Created Dataset ")
        
        
# rotation Part

def RotateMap(size=size):
    os.chdir(Rdirectory)
    for i in trange(size,desc='Rotating images'):
        path = path_rotate+ '/RotatedMaps%d.png' %i
        Nimage = cv2.imread(path)
        #imgr180 = imutils.rotate_bound(Nimage,angle=180)
        Angle = CreateAngle(q0,q1,q2,q3)
        imgr = imutils.rotate_bound(Nimage,angle=-180-(Angle[i]))#Read Angle from the H5 file to get the vehicle rotation
        filename ='%d.png'%i
        cv2.imwrite(filename, imgr) 
    print('Successfully rotated')

#Cropping Part

def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))


def Crop_image(size=size):
   
    os.chdir(path_crop)
    for i in trange(size,desc='Cropping images'):
        im = Image.open(path_crop+'/RotatedMaps%d.png'%i)
        im_new = crop_center(im, 160, 160)
        im_new.save(path_cropped+'/%d.png'%i, quality=100)
    print('Successfully Cropped')
    
########################FUNCTIONS FOR INDIVIDUAL SAVING , ROTATION AND CROPPING##############################
    
def Save_Maps_Sep(Lat,Long,DIFF_LAT,DIFF_LONG,i):
#for i in trange(size,desc='Creating images'): #metres to latitude
    North = Lat-DIFF_LAT                           #0.00121
    South = Lat+DIFF_LAT                             #0.0008
    East  = Long-DIFF_LONG                            #0.00121
    West  = Long+DIFF_LONG                             #0.0017
    fig=get_fig(bbox=(North,South,East,West),size_inches=(6,6))
    fig.savefig(Sep_path_save+'Save%d.png'%i, bbox_inches='tight', pad_inches = 0)
#print("Successfully Created Dataset ")
        

def RotateMap_Sep(i):
    os.chdir(Sep_Rdirectory)
    Nimage = cv2.imread(Sep_path_Rotate+ '/Save%d.png' %i)
    Angle = CreateAngle(q0,q1,q2,q3)
    imgr = imutils.rotate_bound(Nimage,angle=-180-(Angle[i]))#Read Angle from the H5 file to get the vehicle rotation
    filename ='Rotate%d.png'%i
    cv2.imwrite(filename, imgr) 
    print('Successfully rotated')
    
def Crop_image_Sep(i):
    os.chdir(Crop_directory)
    im = Image.open(Crop_directory+ '/Rotate%d.png'%i)
    im_new = crop_center(im, 160, 160)
    im_new.save(Crop_save_path +'/Cropped%d.png'%i, quality=100)
    print('Successfully Cropped')
    

In [ ]:
#For creating just GPS datasets
#Noise_Long,Noise_Lat=Add_Noise(0.0000150,Long,Lat)
#GRoad,GBuilding,BBOX=BBOX_Generation(Noise_Lat,Noise_Long)

#Create a bbox of 80m
#DIFF_LONG,DIFF_LAT=Convert_To_M(Lat,Long)

#Save_Maps(size,Noise_Lat,Noise_Long,DIFF_LAT,DIFF_LONG)
#RotateMap(size)
#Crop_image(size)
#fig = get_fig()
#fig

In [ ]:
Noise_Long,Noise_Lat=Add_Noise(Long,Lat,Noise=0.0000015) #generating noise in Lat and Long

#Read Lat and Long and generate BBOX for roads and buildings
GRoad,GBuilding,BBOX=BBOX_Generation(Noise_Lat,Noise_Long)

#Create a bbox of 80m
DIFF_LONG,DIFF_LAT=Convert_To_M(Lat,Long)
AngleOFROTATIOn = CreateAngle(q0,q1,q2,q3,size)#Generat ing Yaw from the 4 quaternion


Radar1Timestamp = np.round(Radar1Timestamp,2)#RadarTime stamps 
GPSTimestamp=np.round(GPSTimestamp,2)#GPS TIMESTAMPS Converting the last two decimal places to the nearest one.


In [ ]:
#for the GPS+Radartimestamps
for i in trange(GPSTimestamp.size):
    if (GPSTimestamp[i]==Radar1Timestamp[:]).any():
        DIFF_LONG,DIFF_LAT=Convert_To_M(Noise_Lat[i],Noise_Long[i])
        Save_Maps_Sep(Noise_Lat[i],Noise_Long[i],DIFF_LAT,DIFF_LONG,i)
        RotateMap_Sep(i)
        Crop_image_Sep(i)        

In [ ]:
#Converting the images to Numpy
images = [cv2.imread(file) for file in natsorted(glob.glob(Crop_save_path+'/*.png'),alg=ns.REAL)]
numpydata =asarray(images)
print(numpydata.shape)


In [45]:
folder_path = r'C:\Users\bjqb7h\Downloads\Thesis2022\GPS DATA\GPS DATA'  # Replace with the actual folder path

# Search for .h5 files in the folder
log_files = glob.glob(folder_path + '/*.h5')

# Extract log names from file paths and remove the .h5 extension
log_names = [os.path.splitext(os.path.basename(log_file))[0] for log_file in log_files]

log_names_formatted = ' '.join(log_names)

print(' '.join(log_names[:100]))

AtCityBMW_Applanix-20220221T160319Z085 AtCityBMW_Applanix-20220221T160419Z905 AtCityBMW_Applanix-20220221T160443Z413 AtCityBMW_Applanix-20220221T160603Z161 AtCityBMW_Applanix-20220221T160706Z153 AtCityBMW_Applanix-20220221T160814Z341 AtCityBMW_Applanix-20220221T161036Z537 AtCityBMW_Applanix-20220221T161110Z753 AtCityBMW_Applanix-20220221T161154Z449 AtCityBMW_Applanix-20220221T161231Z777 AtCityBMW_Applanix-20220221T161323Z017 AtCityBMW_Applanix-20220221T161358Z594 AtCityBMW_Applanix-20220221T161447Z597 AtCityBMW_Applanix-20220221T161525Z377 AtCityBMW_Applanix-20220221T161730Z481 AtCityBMW_Applanix-20220221T162035Z710 AtCityBMW_Applanix-20220221T162133Z957 AtCityBMW_Applanix-20220221T162222Z237 AtCityBMW_Applanix-20220221T162325Z573 AtCityBMW_Applanix-20220228T141708Z393 AtCityBMW_Applanix-20220228T142342Z338 AtCityBMW_Applanix-20220228T142501Z755 AtCityBMW_Applanix-20220228T142709Z385 AtCityBMW_Applanix-20220228T142815Z329 AtCityBMW_Applanix-20220228T142959Z213 AtCityBMW_Applanix-202202

In [ ]:
#Converting the numpy to HDF5
with h5py.File(hDF5_PATH+'/'+ Logname+'.hdf5', 'w') as hf:
    grp = hf.create_group(Logname)
    dataset = grp.create_dataset('Image data', data=numpydata)

In [ ]:
#delete the images in the save folder
for f in os.listdir(dir0):
    os.remove(os.path.join(dir0, f))
    


In [ ]:
#delete the images in the ROTATE folder
for f1 in os.listdir(dir1):
    os.remove(os.path.join(dir1,f1))

In [ ]:
#delete the images in the CROP folder
for f in os.listdir(dir2):
    os.remove(os.path.join(dir2, f))